In [1318]:
import pandas as pd
import numpy as np
from scipy import spatial
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial import distance
import string
from re import search
 
# Create initial embeddings dictionary
def get_embeddings_dict(embeddings_dict):
    # Open and create dict
    with open(directory + "glove.6B.50d.txt", 'r', encoding="utf-8") as f:
        for line in f:
            # Split line by space
            values = line.split()
            # Get word
            word = values[0]
            # Make array for embeddings
            vector = np.asarray(values[1:], "float32")
            # Update dict
            embeddings_dict[word] = vector
    return(embeddings_dict)

# Create idfs
def get_df_idf():
    word_list = list(line_items["canonical_line_item_name"])
    no_integers = [x for x in word_list if not isinstance(x, int)]
    stemmed = [[stem(word) for word in sentence.split(" ")] for sentence in word_list]

    # Instantiate CountVectorizer()
    cv=CountVectorizer()

    # Generate word counts
    word_count_vector=cv.fit_transform(no_integers)

    # Transform to idf values
    tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
    tfidf_transformer.fit(word_count_vector)

    # Create idf values df
    df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(),columns=["idf_weights"])
    return(df_idf)

# Finds embeddings for each cannonical
def get_canonical_embeddings(line_items):
    # Create empty matrix
    embedding = np.mat([0] * em_dim * len(line_items))
    embedding = embedding.reshape(len(line_items), em_dim)
    line_items = pd.concat([line_items, pd.DataFrame(embedding)], axis=1)

    for i in range(len(line_items)):
        p = 0
        # Removes hourly services as many canonicals add them in addition to line item names
        line = line_items["canonical_line_item_name"][i].translate(str.maketrans('', '', string.punctuation)).replace("Hourly Services:", "").split()
        for word in line:
            # Don't include words that don't have embeddings
            try:
                # Get idf score for word
                idf = df_idf.loc[word.lower()]
                length = len(line_items["canonical_line_item_name"][i].split())
                # Creates weight which favors earlier words
                weight = (length - p)/length
                # Adds word embedding times weight and id to existing sentence embedding
                line_items.loc[[i], line_items.columns[2:]] = line_items.loc[[i], line_items.columns[2:]].add(embeddings_dict[word.lower()]*float(idf)*weight)
                p += 1
            except:
                pass
    return(line_items)

# Makes embedding for line
def make_embedding(line, q):
    p = 0
    embedding = 0
    for word in line:
        try:
            idf = df_idf.loc[word.lower()]
            length = len(line)
            weight = (length - p)/length
            embedding += embeddings_dict[word.lower()]*float(idf)*weight
            p += 1
        except:
                pass
    return(embedding)

# Finds distance between embeddings
def get_distance(embedding, q, same_vendor):
    min_distance = 100000000
    min_distance2 = 100000000
    cat = ""
    i = 0
    for row in same_vendor:
        distance = np.linalg.norm(embedding-row[1][2:])
        if distance < min_distance:
            min_distance2 = min_distance
            min_distance = distance
            cat = i
        i += 1
    return(cat, min_distance, min_distance2)

# Finds accuracy on training data
def get_train_acc():
    correct = [0] * len(train)
    for q in range(len(train)):
        match = 0
        # Find all canonical options for vendor
        vendor_rows = line_items["canonical_line_item_name"].loc[line_items["canonical_vendor_name"] == train["canonical_vendor_name"][q]]
        # Select if there's only one option
        if len(vendor_rows) == 1:
            estimate = vendor_rows.iloc[0]
            match = 1
            # See if estimate is correct
            if estimate == train.loc[q, "canonical_line_item_name"]:
                correct[q] = 1
        if match == 1:
            continue   
            
        # See if canonical is identical to line item name
        for row in line_items["canonical_line_item_name"].loc[line_items["canonical_vendor_name"] == train["canonical_vendor_name"][q]]:
            if row == train.loc[q, "line_item_name"]:
                estimate = row
                match = 1
                if estimate == train.loc[q, "canonical_line_item_name"]:
                    correct[q] = 1

        if match == 1:
            continue  
        
        # Some cases didn't work well with embeddings, a few lines of logic were used to help
        # Filter cases for Amelia Willson
        if train["canonical_vendor_name"][q] == "Amelia Willson":
            if search("900-1,500 words", train["line_item_description"].loc[q]) or search("900-1,500 words", train["line_item_name"].loc[q]):
                match = 1
                estimate = "900-1,500 words"
            elif search("1,500-2,000 words", train["line_item_description"].loc[q]) or search("1,500-2,000 words", train["line_item_name"].loc[q]):
                match = 1
                estimate = "1,500-2,000 words"
            else:
                match = 1
                estimate = "Trial Assignment"
            if estimate == train.loc[q, "canonical_line_item_name"]:
                correct[q] = 1
        
        # Filter cases for Graphite Financial
        if train["canonical_vendor_name"][q] == "Graphite Financial":
            if search("Discounts / Credits Acct", train["line_item_name"].loc[q]):
                match = 1
                estimate = "Discounts / Credits Acct (e)"
            # Make sure line_item_description exists
            elif str(train["line_item_description"].loc[q]) != "nan":
                if search("PROJECT", str(train["line_item_description"].loc[q])):
                    match = 1
                    estimate = "Hourly Services: Projects"
                elif search("Accounting:Core_Accounting_Service", train["line_item_name"].loc[q]):
                    match = 1
                    estimate = "Hourly Services: Tasks"
            if search("Strategic Finance Team", train["line_item_name"].loc[q]):
                match = 1
                estimate = "Hourly Services: Strategic Finance Team"  
            if estimate == train.loc[q, "canonical_line_item_name"]:
                correct[q] = 1
                
        # Filter cases for Maddie Shepherd
        # If name isn't identical to canonical, then its a blog post
        if train["canonical_vendor_name"][q] == "Maddie Shepherd":
            match = 1
            estimate = "Blog Post"  
            if estimate == train.loc[q, "canonical_line_item_name"]:
                correct[q] = 1

        # Filter cases for Andersen Tax
        if train["canonical_vendor_name"][q] == "Andersen Tax":
            if search("Director", train["line_item_description"].loc[q]):
                match = 1
                estimate = "Hourly Services: Legal Services"
            else:
                match = 1
                estimate = "Hourly Services: Tax Services"
            if estimate == train.loc[q, "canonical_line_item_name"]:
                correct[q] = 1
                
        # Filter cases for Daversa Partners
        if train["canonical_vendor_name"][q] == "Daversa Partners":
            match = 1
            estimate = "Retainer: " + train["line_item_name"].loc[q] 
            if estimate == train.loc[q, "canonical_line_item_name"]:
                correct[q] = 1

        # Filter cases for Westmont Associates
        if train["canonical_vendor_name"][q] == "Westmont Associates":
            if search("Buzzy", train["line_item_description"].loc[q]):
                match = 1
                estimate = "Expenses: Buzzy P&C and Surplus Line Renewal"
            # All others are Expenses: Filing Fees
            elif train["line_item_name"].loc[q] == "Expenses":
                match = 1
                estimate = "Expenses: Filing Fees"
            elif train["line_item_name"].loc[q] == "Flat Fee":
                match = 1
                estimate = "Non-Hourly Services: A&A"
            if estimate == train.loc[q, "canonical_line_item_name"]:
                correct[q] = 1
                
        # Filter cases for Xiamen ZhiZhi Tech
        if train["canonical_vendor_name"][q] == "Xiamen ZhiZhi Tech":
            if search("Misc", train["line_item_name"].loc[q]):
                match = 1
                estimate = "Misc. expenses"
            if estimate == train.loc[q, "canonical_line_item_name"]:
                correct[q] = 1
        if match == 1:
            continue

        # Remove punctuation and split by word
        line = str(train.loc[q, "line_item_name"]).translate(str.maketrans('', '', string.punctuation)).split()
        # Create embedding for line
        embedding = make_embedding(line, q)
        # Find canonical options for vendor
        vendor_rows = line_items.loc[line_items["canonical_vendor_name"] == train["canonical_vendor_name"][q]].iterrows() 
        # Find distance between line embedding and canonical possibilities
        cat, min_distance, min_distance2 = get_distance(embedding, q, vendor_rows)

        # Make sure its accurate and clear winner
        # Next phases use line_item_description, need to make sure it exists
        if (min_distance < 25 and min_distance2 - min_distance > 5) or str(train.loc[0, "line_item_description"]) == 'nan':
            # Estimate is closest option
            estimate = line_items["canonical_line_item_name"].loc[line_items["canonical_vendor_name"] == train["canonical_vendor_name"][q]].iloc[cat]
            # See if correct
            if estimate == train.loc[q, "canonical_line_item_name"]:
                correct[q] = 1

        # Run second phase looking at line_item_name/line_item_description combo
        else:
            embedding = 0
            p = 0
            # Concat phrases
            line = str(train.loc[q, "line_item_name"]) + " " + str(train.loc[q, "line_item_description"])
            # Remove punctuation
            line = line.translate(str.maketrans('', '', string.punctuation)).split()
            # Create embedding for line
            embedding = make_embedding(line, q)
            # Find canonical options for vendor
            vendor_rows = line_items.loc[line_items["canonical_vendor_name"] == train["canonical_vendor_name"][q]].iterrows()
            # Find distance between line description embedding and canonical possibilities
            cat, min_distance, min_distance2 = get_distance(embedding, q, vendor_rows)

            # Make sure its accurate and clear winner
            if (min_distance < 40 and min_distance2 - min_distance > 5)or str(train.loc[0, "line_item_description"]) == 'nan':
                # Estimate is closest option
                estimate = line_items["canonical_line_item_name"].loc[line_items["canonical_vendor_name"] == train["canonical_vendor_name"][q]].iloc[cat]
                # See if correct
                if estimate == train.loc[q, "canonical_line_item_name"]:
                    correct[q] = 1
            # Run third phase only using line_item_description
            else:
                embedding = 0
                p = 0
                # Remove punctuation
                line = str(train.loc[q, "line_item_description"])
                # Remove punctuation
                line = line.translate(str.maketrans('', '', string.punctuation)).split()
                # Create embedding for line
                embedding = make_embedding(line, q)
                # Find canonical options for vendor
                vendor_rows = line_items.loc[line_items["canonical_vendor_name"] == train["canonical_vendor_name"][q]].iterrows()
                # Find distance between line description embedding and canonical possibilities
                cat, min_distance, min_distance2 = get_distance(embedding, q, vendor_rows)
                
                # Make sure its accurate and clear winner
                if (min_distance < 20 and min_distance2 - min_distance > 2)or str(train.loc[0, "line_item_description"]) == 'nan':
                    # Estimate is closest option
                    estimate = line_items["canonical_line_item_name"].loc[line_items["canonical_vendor_name"] == train["canonical_vendor_name"][q]].iloc[cat]
                    # See if correct
                    if estimate == train.loc[q, "canonical_line_item_name"]:
                        correct[q] = 1
                        
                # If all three phases produce no clear estimate, none will be selected
                else:
                    correct[q] = None

    print("Correct: " + str(correct.count(1)))
    print("Incorrect: " + str(correct.count(0)))
    print("Unsure: " + str(correct.count(None)))
    return(correct)

# Fills in evaldf
def fill_evaldf(evaldf):
    correct = [0] * len(evaldf)
    for q in range(len(evaldf)):
        match = 0
        # Find all canonical options for vendor
        vendor_rows = line_items["canonical_line_item_name"].loc[line_items["canonical_vendor_name"] == evaldf["canonical_vendor_name"][q]]
        # Select if there's only one option
        if len(vendor_rows) == 1:
            match = 1
            evaldf["canonical_line_item_name"].iloc[q] = vendor_rows.iloc[0]
        if match == 1:
            continue   
            
        # See if canonical is identical to line item name
        for row in line_items["canonical_line_item_name"].loc[line_items["canonical_vendor_name"] == evaldf["canonical_vendor_name"][q]]:
            if row == evaldf.loc[q, "line_item_name"]:
                evaldf["canonical_line_item_name"].iloc[q] = row
                match = 1

        if match == 1:
            continue  
        
        # Some cases didn't work well with embeddings, a few lines of logic were used to help
        # Filter cases for Amelia Willson
        if evaldf["canonical_vendor_name"][q] == "Amelia Willson":
            if search("900-1,500 words", evaldf["line_item_description"].loc[q]) or search("900-1,500 words", evaldf["line_item_name"].loc[q]):
                match = 1
                evaldf["canonical_line_item_name"].iloc[q] = "900-1,500 words"
            elif search("1,500-2,000 words", evaldf["line_item_description"].loc[q]) or search("1,500-2,000 words", evaldf["line_item_name"].loc[q]):
                match = 1
                evaldf["canonical_line_item_name"].iloc[q] = "1,500-2,000 words"
            else:
                match = 1
                evaldf["canonical_line_item_name"].iloc[q] = "Trial Assignment"
        
        # Filter cases for Graphite Financial
        if evaldf["canonical_vendor_name"][q] == "Graphite Financial":
            if search("Discounts / Credits Acct", evaldf["line_item_name"].loc[q]):
                match = 1
                evaldf["canonical_line_item_name"].iloc[q] = "Discounts / Credits Acct (e)"
            # Make sure line_item_description exists
            elif str(evaldf["line_item_description"].loc[q]) != "nan":
                if search("PROJECT", str(evaldf["line_item_description"].loc[q])):
                    match = 1
                    evaldf["canonical_line_item_name"].iloc[q] = "Hourly Services: Projects"
                elif search("Accounting:Core_Accounting_Service", evaldf["line_item_name"].loc[q]):
                    match = 1
                    evaldf["canonical_line_item_name"].iloc[q] = "Hourly Services: Tasks"
            if search("Strategic Finance Team", evaldf["line_item_name"].loc[q]):
                match = 1
                evaldf["canonical_line_item_name"].iloc[q] = "Hourly Services: Strategic Finance Team" 
                
        # Filter cases for Maddie Shepherd
        # If name isn't identical to canonical, then its a blog post
        if evaldf["canonical_vendor_name"][q] == "Maddie Shepherd":
            match = 1
            evaldf["canonical_line_item_name"].iloc[q] = "Blog Post"  

        # Filter cases for Andersen Tax
        if evaldf["canonical_vendor_name"][q] == "Andersen Tax":
            if search("Director", evaldf["line_item_description"].loc[q]):
                match = 1
                evaldf["canonical_line_item_name"].iloc[q] = "Hourly Services: Legal Services"
            else:
                match = 1
                evaldf["canonical_line_item_name"].iloc[q] = "Hourly Services: Tax Services"
                
        # Filter cases for Daversa Partners
        if evaldf["canonical_vendor_name"][q] == "Daversa Partners":
            match = 1
            evaldf["canonical_line_item_name"].iloc[q] = "Retainer: " + evaldf["line_item_name"].loc[q] 

        # Filter cases for Westmont Associates
        if evaldf["canonical_vendor_name"][q] == "Westmont Associates":
            # Make sure line_item_description exists
            if str(evaldf["line_item_description"].loc[q]) != "nan":
                if search("Buzzy", evaldf["line_item_description"].loc[q]):
                    match = 1
                    evaldf["canonical_line_item_name"].iloc[q] = "Expenses: Buzzy P&C and Surplus Line Renewal"
                # All others are Expenses: Filing Fees    
                elif evaldf["line_item_name"].loc[q] == "Expenses":
                    match = 1
            # All others are Expenses: Filing Fees
            if evaldf["line_item_name"].loc[q] == "Expenses":
                match = 1
                evaldf["canonical_line_item_name"].iloc[q] = "Expenses: Filing Fees"
            elif evaldf["line_item_name"].loc[q] == "Flat Fee":
                match = 1
                evaldf["canonical_line_item_name"].iloc[q] = "Non-Hourly Services: A&A"

                
        # Filter cases for Xiamen ZhiZhi Tech
        if evaldf["canonical_vendor_name"][q] == "Xiamen ZhiZhi Tech":
            if search("Misc", evaldf["line_item_name"].loc[q]):
                match = 1
                evaldf["canonical_line_item_name"].iloc[q] = "Misc. expenses"
        if match == 1:
            continue

        # Remove punctuation and split by word
        line = str(evaldf.loc[q, "line_item_name"]).translate(str.maketrans('', '', string.punctuation)).split()
        # Create embedding for line
        embedding = make_embedding(line, q)
        # Find canonical options for vendor
        vendor_rows = line_items.loc[line_items["canonical_vendor_name"] == evaldf["canonical_vendor_name"][q]].iterrows() 
        # Find distance between line embedding and canonical possibilities
        cat, min_distance, min_distance2 = get_distance(embedding, q, vendor_rows)

        # Make sure its accurate and clear winner
        # Next phases use line_item_description, need to make sure it exists
        if (min_distance < 25 and min_distance2 - min_distance > 5) or str(evaldf.loc[0, "line_item_description"]) == 'nan':
            # Estimate is closest option
            evaldf["canonical_line_item_name"].iloc[q] = line_items["canonical_line_item_name"].loc[line_items["canonical_vendor_name"] == evaldf["canonical_vendor_name"][q]].iloc[cat]

        # Run second phase looking at line_item_name/line_item_description combo
        else:
            embedding = 0
            p = 0
            # Concat phrases
            line = str(evaldf.loc[q, "line_item_name"]) + " " + str(evaldf.loc[q, "line_item_description"])
            # Remove punctuation
            line = line.translate(str.maketrans('', '', string.punctuation)).split()
            # Create embedding for line
            embedding = make_embedding(line, q)
            # Find canonical options for vendor
            vendor_rows = line_items.loc[line_items["canonical_vendor_name"] == evaldf["canonical_vendor_name"][q]].iterrows()
            # Find distance between line description embedding and canonical possibilities
            cat, min_distance, min_distance2 = get_distance(embedding, q, vendor_rows)

            # Make sure its accurate and clear winner
            if (min_distance < 40 and min_distance2 - min_distance > 5) or str(evaldf.loc[0, "line_item_description"]) == 'nan':
                # Estimate is closest option
                evaldf["canonical_line_item_name"].iloc[q] = line_items["canonical_line_item_name"].loc[line_items["canonical_vendor_name"] == evaldf["canonical_vendor_name"][q]].iloc[cat]
    
            # Run third phase only using line_item_description
            else:
                embedding = 0
                p = 0
                # Remove punctuation
                line = str(evaldf.loc[q, "line_item_description"])
                # Remove punctuation
                line = line.translate(str.maketrans('', '', string.punctuation)).split()
                # Create embedding for line
                embedding = make_embedding(line, q)
                # Find canonical options for vendor
                vendor_rows = line_items.loc[line_items["canonical_vendor_name"] == evaldf["canonical_vendor_name"][q]].iterrows()
                # Find distance between line description embedding and canonical possibilities
                cat, min_distance, min_distance2 = get_distance(embedding, q, vendor_rows)
                
                # Make sure its accurate and clear winner
                if (min_distance < 20 and min_distance2 - min_distance > 2)or str(evaldf.loc[0, "line_item_description"]) == 'nan':
                    # Estimate is closest option
                    evaldf["canonical_line_item_name"].iloc[q] = line_items["canonical_line_item_name"].loc[line_items["canonical_vendor_name"] == evaldf["canonical_vendor_name"][q]].iloc[cat]
                        
                # If all three phases produce no clear estimate, none will be selected
                else:
                    evaldf["canonical_line_item_name"].iloc[q] = None

    return(evaldf)

In [1336]:
import warnings

warnings.simplefilter('always', UserWarning)

In [1337]:
# Load Excel Files
directory = "/Users/philliphicks/Desktop/Glean Project/"
train = pd.read_excel(directory + "question-python-data-science-project-mwsr7tgbeo-mapping_challenge.xlsx", sheet_name = "train")
evaldf = pd.read_excel(directory + "question-python-data-science-project-mwsr7tgbeo-mapping_challenge.xlsx", sheet_name = "eval")
line_items = pd.read_excel(directory + "question-python-data-science-project-mwsr7tgbeo-mapping_challenge.xlsx", sheet_name = "canonical_line_item_table")

embeddings_dict = {}
# Download the pretrained 6B vectors at https://nlp.stanford.edu/projects/glove/
embeddings_dict = get_embeddings_dict(embeddings_dict)
# Get idf data frame
df_idf = get_df_idf()
# Finds embeddings for each canonical
line_items = get_canonical_embeddings(line_items)
# Finds accuracy for training set
correct = get_train_acc()
# Fills in evaldf
evaldf = fill_evaldf(evaldf)
# Writes to csv
evaldf.to_csv(directory + "answers.csv")

Correct: 530
Incorrect: 38
Unsure: 91


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [1335]:
line_items = get_canonical_embeddings(line_items)

In [1333]:
embeddings_dict = get_embeddings_dict(embeddings_dict)

In [1303]:

correct = [0] * len(evaldf)
for q in range(len(evaldf)):
    
    match = 0
    # Find all canonical options for vendor
    vendor_rows = line_items["canonical_line_item_name"].loc[line_items["canonical_vendor_name"] == evaldf["canonical_vendor_name"][q]]
    # Select if there's only one option
    if len(vendor_rows) == 1:
        match = 1
        evaldf["canonical_line_item_name"].iloc[q] = vendor_rows.iloc[0]
    if match == 1:
        continue   

    # See if canonical is identical to line item name
    for row in line_items["canonical_line_item_name"].loc[line_items["canonical_vendor_name"] == evaldf["canonical_vendor_name"][q]]:
        if row == evaldf.loc[q, "line_item_name"]:
            evaldf["canonical_line_item_name"].iloc[q] = row
            match = 1

    if match == 1:
        continue  

    # Some cases didn't work well with embeddings, a few lines of logic were used to help
    # Filter cases for Amelia Willson
    if evaldf["canonical_vendor_name"][q] == "Amelia Willson":
        if search("900-1,500 words", evaldf["line_item_description"].loc[q]) or search("900-1,500 words", evaldf["line_item_name"].loc[q]):
            match = 1
            evaldf["canonical_line_item_name"].iloc[q] = "900-1,500 words"
        elif search("1,500-2,000 words", evaldf["line_item_description"].loc[q]) or search("1,500-2,000 words", evaldf["line_item_name"].loc[q]):
            match = 1
            evaldf["canonical_line_item_name"].iloc[q] = "1,500-2,000 words"
        else:
            match = 1
            evaldf["canonical_line_item_name"].iloc[q] = "Trial Assignment"

    # Filter cases for Graphite Financial
    if evaldf["canonical_vendor_name"][q] == "Graphite Financial":
        if search("Discounts / Credits Acct", evaldf["line_item_name"].loc[q]):
            match = 1
            evaldf["canonical_line_item_name"].iloc[q] = "Discounts / Credits Acct (e)"
        # Make sure line_item_description exists
        elif str(evaldf["line_item_description"].loc[q]) != "nan":
            if search("PROJECT", str(evaldf["line_item_description"].loc[q])):
                match = 1
                evaldf["canonical_line_item_name"].iloc[q] = "Hourly Services: Projects"
            elif search("Accounting:Core_Accounting_Service", evaldf["line_item_name"].loc[q]):
                match = 1
                evaldf["canonical_line_item_name"].iloc[q] = "Hourly Services: Tasks"
        if search("Strategic Finance Team", evaldf["line_item_name"].loc[q]):
            match = 1
            evaldf["canonical_line_item_name"].iloc[q] = "Hourly Services: Strategic Finance Team" 

    # Filter cases for Maddie Shepherd
    # If name isn't identical to canonical, then its a blog post
    if evaldf["canonical_vendor_name"][q] == "Maddie Shepherd":
        match = 1
        evaldf["canonical_line_item_name"].iloc[q] = "Blog Post"  

    # Filter cases for Andersen Tax
    if evaldf["canonical_vendor_name"][q] == "Andersen Tax":
        if search("Director", evaldf["line_item_description"].loc[q]):
            match = 1
            evaldf["canonical_line_item_name"].iloc[q] = "Hourly Services: Legal Services"
        else:
            match = 1
            evaldf["canonical_line_item_name"].iloc[q] = "Hourly Services: Tax Services"

    # Filter cases for Daversa Partners
    if evaldf["canonical_vendor_name"][q] == "Daversa Partners":
        match = 1
        evaldf["canonical_line_item_name"].iloc[q] = "Retainer: " + evaldf["line_item_name"].loc[q] 

    # Filter cases for Westmont Associates
    if evaldf["canonical_vendor_name"][q] == "Westmont Associates":
        # Make sure line_item_description exists
        if str(evaldf["line_item_description"].loc[q]) != "nan":
            if search("Buzzy", evaldf["line_item_description"].loc[q]):
                match = 1
                evaldf["canonical_line_item_name"].iloc[q] = "Expenses: Buzzy P&C and Surplus Line Renewal"
            # All others are Expenses: Filing Fees    
            elif evaldf["line_item_name"].loc[q] == "Expenses":
                match = 1
        # All others are Expenses: Filing Fees
        if evaldf["line_item_name"].loc[q] == "Expenses":
            match = 1
            evaldf["canonical_line_item_name"].iloc[q] = "Expenses: Filing Fees"
        elif evaldf["line_item_name"].loc[q] == "Flat Fee":
            match = 1
            evaldf["canonical_line_item_name"].iloc[q] = "Non-Hourly Services: A&A"

    # Filter cases for Xiamen ZhiZhi Tech
    if evaldf["canonical_vendor_name"][q] == "Xiamen ZhiZhi Tech":
        if search("Misc", evaldf["line_item_name"].loc[q]):
            match = 1
            evaldf["canonical_line_item_name"].iloc[q] = "Misc. expenses"
    if match == 1:
        continue

    # Remove punctuation and split by word
    line = str(evaldf.loc[q, "line_item_name"]).translate(str.maketrans('', '', string.punctuation)).split()
    # Create embedding for line
    embedding = make_embedding(line, q)
    # Find canonical options for vendor
    vendor_rows = line_items.loc[line_items["canonical_vendor_name"] == evaldf["canonical_vendor_name"][q]].iterrows() 
    # Find distance between line embedding and canonical possibilities
    cat, min_distance, min_distance2 = get_distance(embedding, q, vendor_rows)

    # Make sure its accurate and clear winner
    # Next phases use line_item_description, need to make sure it exists
    if (min_distance < 25 and min_distance2 - min_distance > 5) or str(evaldf.loc[0, "line_item_description"]) == 'nan':
        # Estimate is closest option
        evaldf["canonical_line_item_name"].iloc[q] = line_items["canonical_line_item_name"].loc[line_items["canonical_vendor_name"] == evaldf["canonical_vendor_name"][q]].iloc[cat]

    # Run second phase looking at line_item_name/line_item_description combo
    else:
        embedding = 0
        p = 0
        # Concat phrases
        line = str(evaldf.loc[q, "line_item_name"]) + " " + str(evaldf.loc[q, "line_item_description"])
        # Remove punctuation
        line = line.translate(str.maketrans('', '', string.punctuation)).split()
        # Create embedding for line
        embedding = make_embedding(line, q)
        # Find canonical options for vendor
        vendor_rows = line_items.loc[line_items["canonical_vendor_name"] == evaldf["canonical_vendor_name"][q]].iterrows()
        # Find distance between line description embedding and canonical possibilities
        cat, min_distance, min_distance2 = get_distance(embedding, q, vendor_rows)

        # Make sure its accurate and clear winner
        if (min_distance < 40 and min_distance2 - min_distance > 5) or str(evaldf.loc[0, "line_item_description"]) == 'nan':
            # Estimate is closest option
            evaldf["canonical_line_item_name"].iloc[q] = line_items["canonical_line_item_name"].loc[line_items["canonical_vendor_name"] == evaldf["canonical_vendor_name"][q]].iloc[cat]

        # Run third phase only using line_item_description
        else:
            embedding = 0
            p = 0
            # Remove punctuation
            line = str(evaldf.loc[q, "line_item_description"])
            # Remove punctuation
            line = line.translate(str.maketrans('', '', string.punctuation)).split()
            # Create embedding for line
            embedding = make_embedding(line, q)
            # Find canonical options for vendor
            vendor_rows = line_items.loc[line_items["canonical_vendor_name"] == evaldf["canonical_vendor_name"][q]].iterrows()
            # Find distance between line description embedding and canonical possibilities
            cat, min_distance, min_distance2 = get_distance(embedding, q, vendor_rows)

            # Make sure its accurate and clear winner
            if (min_distance < 20 and min_distance2 - min_distance > 2)or str(evaldf.loc[0, "line_item_description"]) == 'nan':
                # Estimate is closest option
                evaldf["canonical_line_item_name"].iloc[q] = line_items["canonical_line_item_name"].loc[line_items["canonical_vendor_name"] == evaldf["canonical_vendor_name"][q]].iloc[cat]

            # If all three phases produce no clear estimate, none will be selected
            else:
                evaldf["canonical_line_item_name"].iloc[q] = None



In [1309]:
evaldf["canonical_line_item_name"].count(None)

296

In [850]:
len(line_items["canonical_line_item_name"][i].split())

3

In [1259]:
def get_canonical_embeddings():
    embedding = np.mat([0] * em_dim * len(line_items))
    embedding = embedding.reshape(len(line_items), em_dim)
    line_items = pd.concat([line_items, pd.DataFrame(embedding)], axis=1)

    for i in range(len(line_items)):
        p = 0
        # Removes hourly services as many canonicals add them in addition to line item names
        line = line_items["canonical_line_item_name"][i].translate(str.maketrans('', '', string.punctuation)).replace("Hourly Services:", "").split()
        for word in line:
            try:
                idf = df_idf.loc[word.lower()]
                length = len(line_items["canonical_line_item_name"][i].split())
                weight = (length - p)/length
                line_items.loc[[i], line_items.columns[2:]] = line_items.loc[[i], line_items.columns[2:]].add(embeddings_dict[word.lower()]*float(idf)*weight)
                p += 1
            except:
                pass
    return(embedding)


In [1320]:
evaldf.to_csv(directory + "answers.csv")

In [1288]:

correct = [0] * len(train)
for q in range(len(train)):
    match = 0
    # Find all canonical options for vendor
    vendor_rows = line_items["canonical_line_item_name"].loc[line_items["canonical_vendor_name"] == train["canonical_vendor_name"][q]]
    # Select if there's only one option
    if len(vendor_rows) == 1:
        estimate = vendor_rows.iloc[0]
        match = 1
        # See if estimate is correct
        if estimate == train.loc[q, "canonical_line_item_name"]:
            correct[q] = 1
    if match == 1:
        continue   

    # See if canonical is identical to line item name
    for row in line_items["canonical_line_item_name"].loc[line_items["canonical_vendor_name"] == train["canonical_vendor_name"][q]]:
        if row == train.loc[q, "line_item_name"]:
            estimate = row
            match = 1
            if estimate == train.loc[q, "canonical_line_item_name"]:
                correct[q] = 1

    if match == 1:
        continue  

    # Some cases didn't work well with embeddings, a few lines of logic were used to help
    # Filter cases for Amelia Willson
    if train["canonical_vendor_name"][q] == "Amelia Willson":
        if search("900-1,500 words", train["line_item_description"].loc[q]) or search("900-1,500 words", train["line_item_name"].loc[q]):
            match = 1
            estimate = "900-1,500 words"
        elif search("1,500-2,000 words", train["line_item_description"].loc[q]) or search("1,500-2,000 words", train["line_item_name"].loc[q]):
            match = 1
            estimate = "1,500-2,000 words"
        else:
            match = 1
            estimate = "Trial Assignment"
        if estimate == train.loc[q, "canonical_line_item_name"]:
            correct[q] = 1

    # Filter cases for Graphite Financial
    if train["canonical_vendor_name"][q] == "Graphite Financial":
        if search("Discounts / Credits Acct", train["line_item_name"].loc[q]):
            match = 1
            estimate = "Discounts / Credits Acct (e)"
        # Make sure line_item_description exists
        elif str(train["line_item_description"].loc[q]) != "nan":
            if search("PROJECT", str(train["line_item_description"].loc[q])):
                match = 1
                estimate = "Hourly Services: Projects"
            elif search("Accounting:Core_Accounting_Service", train["line_item_name"].loc[q]):
                match = 1
                estimate = "Hourly Services: Tasks"
        if search("Strategic Finance Team", train["line_item_name"].loc[q]):
            match = 1
            estimate = "Hourly Services: Strategic Finance Team"  
        if estimate == train.loc[q, "canonical_line_item_name"]:
            correct[q] = 1

    # Filter cases for Maddie Shepherd
    # If name isn't identical to canonical, then its a blog post
    if train["canonical_vendor_name"][q] == "Maddie Shepherd":
        match = 1
        estimate = "Blog Post"  
        if estimate == train.loc[q, "canonical_line_item_name"]:
            correct[q] = 1

    # Filter cases for Andersen Tax
    if train["canonical_vendor_name"][q] == "Andersen Tax":
        if search("Director", train["line_item_description"].loc[q]):
            match = 1
            estimate = "Hourly Services: Legal Services"
        else:
            match = 1
            estimate = "Hourly Services: Tax Services"
        if estimate == train.loc[q, "canonical_line_item_name"]:
            correct[q] = 1

    # Filter cases for Daversa Partners
    if train["canonical_vendor_name"][q] == "Daversa Partners":
        match = 1
        estimate = "Retainer: " + train["line_item_name"].loc[q] 
        if estimate == train.loc[q, "canonical_line_item_name"]:
            correct[q] = 1

    # Filter cases for Westmont Associates
    if train["canonical_vendor_name"][q] == "Westmont Associates":
        if search("Buzzy", train["line_item_description"].loc[q]):
            match = 1
            estimate = "Expenses: Buzzy P&C and Surplus Line Renewal"
        # All others are Expenses: Filing Fees
        elif train["line_item_name"].loc[q] == "Expenses":
            match = 1
            estimate = "Expenses: Filing Fees"
        elif train["line_item_name"].loc[q] == "Flat Fee":
            match = 1
            estimate = "Non-Hourly Services: A&A"
        if estimate == train.loc[q, "canonical_line_item_name"]:
            correct[q] = 1

    # Filter cases for Xiamen ZhiZhi Tech
    if train["canonical_vendor_name"][q] == "Xiamen ZhiZhi Tech":
        if search("Misc", train["line_item_name"].loc[q]):
            match = 1
            estimate = "Misc. expenses"
        if estimate == train.loc[q, "canonical_line_item_name"]:
            correct[q] = 1
    if match == 1:
        continue

    # Remove punctuation and split by word
    line = str(train.loc[q, "line_item_name"]).translate(str.maketrans('', '', string.punctuation)).split()
    # Create embedding for line
    embedding = make_embedding(line, q)
    # Find canonical options for vendor
    vendor_rows = line_items.loc[line_items["canonical_vendor_name"] == train["canonical_vendor_name"][q]].iterrows() 
    # Find distance between line embedding and canonical possibilities
    cat, min_distance, min_distance2 = get_distance(embedding, q, vendor_rows)

    # Make sure its accurate and clear winner
    # Next phases use line_item_description, need to make sure it exists
    if (min_distance < 25 and min_distance2 - min_distance > 5) or str(train.loc[0, "line_item_description"]) == 'nan':
        # Estimate is closest option
        estimate = line_items["canonical_line_item_name"].loc[line_items["canonical_vendor_name"] == train["canonical_vendor_name"][q]].iloc[cat]
        # See if correct
        if estimate == train.loc[q, "canonical_line_item_name"]:
            correct[q] = 1

    # Run second phase looking at line_item_name/line_item_description combo
    else:
        embedding = 0
        p = 0
        # Concat phrases
        line = str(train.loc[q, "line_item_name"]) + " " + str(train.loc[q, "line_item_description"])
        # Remove punctuation
        line = line.translate(str.maketrans('', '', string.punctuation)).split()
        # Create embedding for line
        embedding = make_embedding(line, q)
        # Find canonical options for vendor
        vendor_rows = line_items.loc[line_items["canonical_vendor_name"] == train["canonical_vendor_name"][q]].iterrows()
        
        # Find distance between line description embedding and canonical possibilities
        cat, min_distance, min_distance2 = get_distance(embedding, q, vendor_rows)

        # Make sure its accurate and clear winner
        if (min_distance < 40 and min_distance2 - min_distance > 5)or str(train.loc[0, "line_item_description"]) == 'nan':
            # Estimate is closest option
            estimate = line_items["canonical_line_item_name"].loc[line_items["canonical_vendor_name"] == train["canonical_vendor_name"][q]].iloc[cat]
            # See if correct
            if estimate == train.loc[q, "canonical_line_item_name"]:
                correct[q] = 1

        else:
            embedding = 0
            p = 0
            # Remove punctuation
            line = str(train.loc[q, "line_item_description"])
            line = line.translate(str.maketrans('', '', string.punctuation)).split()

            embedding = make_embedding(line, q)

            min_distance = 100000000
            cat = ""
            i = 0
            # Find canonical options for vendor
            vendor_rows = line_items.loc[line_items["canonical_vendor_name"] == train["canonical_vendor_name"][q]].iterrows()
            cat, min_distance, min_distance2 = get_distance(embedding, q, vendor_rows)

            if (min_distance < 20 and min_distance2 - min_distance > 2)or str(train.loc[0, "line_item_description"]) == 'nan':
                estimate = line_items["canonical_line_item_name"].loc[line_items["canonical_vendor_name"] == train["canonical_vendor_name"][q]].iloc[cat]
                if estimate == train.loc[q, "canonical_line_item_name"]:
                    correct[q] = 1
            else:
                correct[q] = None

print(correct.count(1))
print(correct.count(0))
print(correct.count(None))

530
38
91


In [1206]:
if train["canonical_vendor_name"][q] == "Graphite Financial":
        if search("Discounts / Credits Acct", train["line_item_name"].loc[q]):
            match = 1
            estimate = "Discounts / Credits Acct (e)"
        elif str(train["line_item_description"].loc[q]) != "nan":
            if search("PROJECT", str(train["line_item_description"].loc[q])):
                match = 1
                estimate = "Hourly Services: Projects"
            elif search("Accounting:Core_Accounting_Service", train["line_item_name"].loc[q]):
                match = 1
                estimate = "Hourly Services: Tasks"
            
        if search("Strategic Finance Team", train["line_item_name"].loc[q]):
            match = 1
            estimate = "Hourly Services: Strategic Finance Team"  
        if estimate == train.loc[q, "canonical_line_item_name"]:
            correct[q] = 1

In [1199]:
embeddings_dict["tina"]

array([-0.78116  ,  0.67316  ,  0.052975 ,  0.96085  ,  0.015799 ,
        1.2616   , -0.65547  ,  0.36868  , -0.29412  , -0.39118  ,
       -0.0085021, -0.13863  , -0.24932  , -0.27744  ,  0.88921  ,
        0.26093  ,  0.095309 , -0.69008  ,  0.74654  ,  0.25551  ,
       -0.32599  ,  0.68659  ,  0.55342  ,  0.7021   ,  0.60021  ,
       -0.0070001, -0.21531  ,  0.10598  ,  0.11562  , -0.97525  ,
        0.29971  ,  0.18012  ,  0.65299  , -0.044404 , -0.55389  ,
        0.2542   , -0.18848  ,  0.014686 ,  0.53876  , -0.62117  ,
        0.66852  ,  0.89929  , -0.64096  , -1.1061   , -0.22891  ,
       -0.23983  , -0.036699 , -1.3094   , -0.3809   ,  0.63955  ],
      dtype=float32)

In [1064]:
embedding = 0
p = 0
line = str(train.loc[q, "line_item_name"]).translate(str.maketrans('', '', string.punctuation)).split()
for word in line:
    try:
        idf = df_idf.loc[word.lower()]
        length = len(line)
        weight = (length - p)/length
        embedding += embeddings_dict[word.lower()]*float(idf)*weight
        p += 1
    except:
            pass
    print(word)

Sam
Web


In [1211]:
def make_embedding(line, q):
    p = 0
    embedding = 0
    for word in line:
        try:
            idf = df_idf.loc[word.lower()]
            length = len(line)
            weight = (length - p)/length
            embedding += embeddings_dict[word.lower()]*float(idf)*weight
            p += 1
        except:
                pass
    return(embedding)

def get_distance(embedding, q, same_vendor):
    min_distance = 100000000
    min_distance2 = 100000000
    cat = ""
    i = 0
    for row in same_vendor:
        distance = np.linalg.norm(embedding-row[1][2:])
        if distance < min_distance:
            min_distance2 = min_distance
            min_distance = distance
            cat = i
        i += 1
    return(cat, min_distance, min_distance2)

In [1202]:
5**2

25

In [1098]:
same_vendor = line_items.loc[line_items["canonical_vendor_name"] == train["canonical_vendor_name"][q]].iterrows()
cat, min_distance, min_distance2 = get_distance(embedding, q, same_vendor)
cat

4

In [914]:
match = 0
for row in line_items["canonical_line_item_name"].loc[line_items["canonical_vendor_name"] == train["canonical_vendor_name"][q]]:
    if row == train.loc[q, "line_item_name"]:
        estimate = row
        match = 1
        if estimate == train.loc[q, "canonical_line_item_name"]:
            correct[q] = 1

    if len(row.split()) == 1 and row in str(train.loc[q, "line_item_name"]):
        estimate = row
        match = 1
        print("hi")
        if estimate == train.loc[q, "canonical_line_item_name"]:
            correct[q] = 1

In [1311]:
output = [idx for idx, element in enumerate(correct) if element == None]
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(evaldf["canonical_line_item_name"])


0                                    Management Services
1                                         Acrobat Pro DC
2      AIEX 96 Pieces Adhesive Poster Tacky Putty Sti...
3      AmazonBasics AAA 1.5 Volt Performance Alkaline...
4               AmazonBasics Mesh Trash Can Waste Basket
5      AmazonFresh Mediterranean Extra Virgin Olive O...
6        Apple 87W USB-C Power Adapter (for MacBook Pro)
7      Apple iPad with Retina Display MD511LL/A (32GB...
8                         Apple iPad with Retina Display
9      Apple iPad with Retina Display MD511LL/A (32GB...
10     Assurant B2B 4YR Kitchen Protection Plan with ...
11     Bose QuietComfort 35 (Series I) Wireless Headp...
12     Calculator,Vilcome 12-Digit Solar Battery Offi...
13     Dell 130-WATT 3-Prong AC Adapter with 6 FT Pow...
14     Logitech K400 Plus Wireless Touch TV Keyboard ...
15     Microsoft Natural Ergonomic Keyboard 4000 for ...
16     Mouthwash Dispenser Mini (White) - For GotFres...
17     Mrs. Meyer's Clean Day H

In [ ]:
evaldf["canonical_line_item_name"].co

In [1317]:
print(sum(x is  None for x in evaldf["canonical_line_item_name"]))

41


In [1157]:
if search("900-1,500 words", train["line_item_description"].loc[81]) or search("900-1,500 words", train["line_item_description"].loc[81]):
    print("y")

y


In [919]:
embedding = 0
for word in str(train.loc[q, "line_item_description"]).split():
    try:
        idf = df_idf.loc[word.lower()]
        embedding += embeddings_dict[word.lower()]*float(idf)
    except:
            pass


min_distance = 100000000
distance_i = {}
cat = ""
i = 0
for row in line_items.loc[line_items["canonical_vendor_name"] == train["canonical_vendor_name"][q]].iterrows():
    distance = np.linalg.norm(embedding-row[1][2:])
    if distance < min_distance:
        min_distance = distance
        cat = i
    distance_i[i] = distance
    i += 1
estimate = line_items["canonical_line_item_name"].loc[line_items["canonical_vendor_name"] == train["canonical_vendor_name"][q]].iloc[cat]
if estimate == train.loc[q, "canonical_line_item_name"]:
    print(1)

1


In [1230]:
embedding = 0
for word in str(train.loc[q, "line_item_description"]).split():
    try:
        idf = df_idf.loc[word.lower()]
        embedding += embeddings_dict[word.lower()]*float(idf)
        print(word)
    except:
            pass

EB


In [1235]:
print(embedding)
line_items.loc[530]
correct[q]

[-1.3160781   1.3273141   2.5891945   5.1898236  -9.923461   -3.5404956
  3.554706   -6.409998   -1.8552787  -2.6395586   0.9418497  -1.3343863
 -8.709962   -4.054713    1.8415971   2.227458   -1.7517083   3.1866908
  1.8881279   6.7013435  -7.071474   -3.5002441   1.9012806   4.165884
 -0.9989556   5.745151   -4.469589   -0.787849    0.8863302  -2.366191
  1.920382    0.878531   -0.88018334 12.287008   -2.7550921  -1.2497851
  8.404604   -1.757723    1.4684924   3.6431408   5.333315   -6.6378927
  1.0982299   0.9485253   1.5623469  -1.6296314   1.3550739   2.0103369
  1.6975766   3.7915235 ]


1

In [834]:
line_items

,canonical_vendor_name,canonical_line_item_name,0,1,2,3,4,5,6,7,...,40,41,42,43,44,45,46,47,48,49
0,10 Minute Ventures,Management Services,1.281185e+05,3.784094e+05,1.964609e+05,2.978463e+05,2.219045e+05,4.194802e+05,3.614978e+05,5.136573e+05,...,1.656515e+04,4.377685e+05,2.773188e+05,5.163942e+05,2.202789e+05,2.412881e+05,5.396405e+05,6.936241e+04,163722.650391,1.450251e+05
1,ACORD,eForms Redistribution,6.021758e+05,4.184985e+05,4.966027e+05,4.223915e+05,7.104521e+04,5.874564e+05,2.557998e+05,1.761689e+05,...,3.761384e+05,7.904928e+03,1.897447e+05,3.368121e+05,1.838358e+05,3.124628e+05,4.729672e+05,1.585414e+05,532915.125000,9.265158e+04
2,Acqcom Digital Marketing,Web Media Fee,6.280344e+05,5.223187e+05,3.383052e+05,9.577094e+05,5.942596e+04,3.021966e+05,5.428117e+05,4.192241e+05,...,1.300908e+05,4.263997e+05,5.725898e+05,6.323712e+04,2.343759e+05,2.634757e+05,5.751207e+05,7.434638e+05,375602.783203,3.598805e+05
3,CSC,DISBURSEMENT/COST - ANNUAL REPORT/TAX RETURN,4.087518e+05,2.097051e+05,4.346557e+05,2.500448e+05,4.114806e+05,5.467892e+05,3.382705e+05,9.068359e+05,...,1.836891e+05,2.940327e+05,4.076276e+05,3.816106e+05,4.513314e+05,1.028191e+05,4.735081e+05,2.705489e+05,365348.890625,4.045292e+05
4,CSC,FOREIGN FILING,2.333249e+05,4.597399e+05,1.330276e+05,2.691590e+05,3.897187e+05,5.035857e+05,2.780180e+05,7.195848e+05,...,2.019144e+05,1.498411e+05,1.304903e+05,8.654233e+04,6.703136e+05,6.535559e+05,1.778693e+05,2.814486e+05,338543.742188,1.550274e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,Microsoft Azure,Storage: General Block Blob LRS Data Stored,8.581907e+05,7.953010e+05,3.728210e+05,6.176481e+05,7.788250e+05,9.434607e+05,7.415660e+05,9.140884e+05,...,5.292033e+05,9.038115e+05,9.982839e+05,1.055094e+06,5.709619e+05,7.692614e+05,1.016787e+06,6.880339e+05,761603.719727,5.047693e+05
541,STIGroup,Agreement Cybersecurity Program Implementation...,1.049399e+06,9.903477e+05,1.162100e+06,1.217856e+06,8.540021e+05,1.321795e+06,1.241556e+06,1.584835e+06,...,1.153964e+06,1.665761e+06,1.075256e+06,1.195187e+06,1.325911e+06,6.175643e+05,9.457804e+05,3.586980e+05,984570.800781,1.222367e+06
542,Staples,"Staples Carpet Chair Mat, 36"" x 48', Crystal C...",7.611665e+05,1.256991e+06,1.600478e+06,1.303988e+06,1.957768e+06,1.527207e+06,3.690420e+05,9.881796e+05,...,7.149057e+05,1.595324e+06,7.000205e+05,6.672301e+05,1.080941e+06,1.475641e+06,1.177388e+06,5.581805e+05,729578.048828,1.654593e+06
543,Staples,Staples Carder Mesh Back Fabric Computer and D...,1.054042e+06,1.285991e+06,1.714092e+06,1.539200e+06,1.395700e+06,1.388437e+06,1.275212e+06,9.877373e+05,...,1.545456e+06,1.314916e+06,1.625727e+06,8.783812e+05,1.082654e+06,1.519711e+06,1.373781e+06,1.153283e+06,665406.825195,1.798067e+06


In [806]:

a = str(train.loc[q, "line_item_description"]).translate(str.maketrans('', '', string.punctuation)).split()
for word in a:
    print(word)

1
Segment
Integration
Contractor
2
Iterable
Integration
